In [1]:
import pandas as pd
import numpy as np
import unicodedata
import re
import pyarrow.parquet as pq
import pyarrow.dataset as ds


### chargement des colonnes nécessaires 

In [2]:
dataset = ds.dataset('C:/Users/Utilisateur/OneDrive/Documents/centrale Lille + sc po/Projet SOGA/ess-numerique-database/data/raw/StockUniteLegale_utf8.parquet', format="parquet")


### Filtrage : ESS + Codes NAF Numérique


In [6]:

codes_naf_numerique = [
    '62.01Z', '62.02A', '62.02B', '62.03Z', 
    '63.11Z', '63.12Z', '58.21Z', '58.29C', '58.29A', '58.29B'
]



In [3]:
# On ne charge que les lignes ESS = 'O' pour économiser la RAM
table = dataset.to_table(
    columns=[
        'siren', 'denominationUniteLegale', 'nomUniteLegale', 'prenom1UniteLegale',
        'activitePrincipaleUniteLegale', 'economieSocialeSolidaireUniteLegale',
        'categorieJuridiqueUniteLegale', 'trancheEffectifsUniteLegale'
    ],
    filter=(ds.field('economieSocialeSolidaireUniteLegale') == 'O')
)

### Ceci filtre en Panda mais seulement après le filtre pour ne pas charger ce que l'on ne veut pas

In [4]:
df_insee = table.to_pandas()

In [7]:
df_filtered = df_insee[df_insee['activitePrincipaleUniteLegale'].isin(codes_naf_numerique)].copy()

In [ ]:
df_filtered.to_csv('../data/processed/ess_numerique_filtered.csv', index=False)
print(f"Filtrage terminé. {len(df_filtered)} acteurs identifiés.")

Filtrage terminé. 3323 acteurs identifiés.


 Maintenant, je souhaiterais dans filtrage comparer le SIREN de ce que j'ai récupéré avec mes filtres avec la base entreprisesess, et ne garder que les structures qui sont dans les 2 docs 

In [12]:
df_base_essfrance = pd.read_csv(
    'C:/Users/Utilisateur/OneDrive/Documents/centrale Lille + sc po/Projet SOGA/ess-numerique-database/data/raw/entreprisesess.csv', 
    sep=',',               # Changement ici : virgule au lieu de point-virgule
    dtype=str,             # On charge tout en texte pour conserver les SIREN intacts
)

In [13]:
print("Colonnes détectées dans ton fichier CSV (entreprisesess.csv) :")
print(df_base_essfrance.columns.tolist())

Colonnes détectées dans ton fichier CSV (entreprisesess.csv) :
['SIREN', "Nom ou raison sociale de l'entreprise", "Famille juridique de l'entreprise", 'Code postal', "Libellé de la commune de l'établissement", "Code du département de l'établissement", "Département de l'établissement", "Région de l'établissement"]


In [14]:
liste_siren_references = df_base_essfrance['SIREN'].unique()

In [15]:
df_base_essfrance.columns = df_base_essfrance.columns.str.replace('"', '').str.strip().str.lower()

In [16]:
liste_siren_references = df_base_essfrance['siren'].unique()

print(f"Colonnes nettoyées dans ton CSV : {df_base_essfrance.columns.tolist()}")
print(f"Nombre de SIREN uniques chargés : {len(liste_siren_references)}")

Colonnes nettoyées dans ton CSV : ['siren', "nom ou raison sociale de l'entreprise", "famille juridique de l'entreprise", 'code postal', "libellé de la commune de l'établissement", "code du département de l'établissement", "département de l'établissement", "région de l'établissement"]
Nombre de SIREN uniques chargés : 1130252


#### Filtrage par code NAF

In [17]:
df_insee_filt = df_insee[df_insee['activitePrincipaleUniteLegale'].isin(codes_naf_numerique)].copy()

#### On s'assure que les deux colonnes sont bien comparables (string et sans espaces)


In [18]:
df_insee_filt['siren'] = df_insee_filt['siren'].astype(str).str.strip()
liste_siren_references = [str(s).strip() for s in liste_siren_references]

In [19]:
df_2filtres = df_insee_filt[df_insee_filt['siren'].isin(liste_siren_references)].copy()

In [21]:
df_2filtres.to_csv('../data/processed/df_filtrage_final.csv', index=False)
print(f"Nombre final de structures croisées : {len(df_2filtres)}")

Nombre final de structures croisées : 1956
